In [8]:
import pandas as pd

# Carica i file TSV
CRISPR_df = pd.read_csv('/home/isacco.cenacchi/data/Tirocinio/out/MAGs_minced_parsed.tsv', delimiter='\t', dtype={'MAG': str, 'contig': str, 'start': int, 'end': int}, index_col=0)
CRISPR_df.rename(columns={'contig': 'Contig', 'start': 'Start', 'end': 'End'}, inplace=True)
Cas_df = pd.read_csv('/home/isacco.cenacchi/data/Tirocinio/samples/Aug19_cas_genes.tsv', delimiter='\t', usecols=['MAG', 'Contig', 'Start', 'End'], dtype={'MAG': str, 'Contig': str, 'Start': int, 'End': int})

# Controllare l'utilizzo della memoria
# print(CRISPR_df.memory_usage(deep=True))
# print(Cas_df.memory_usage(deep=True))

# Creo un DataFrame con i dati dei CRISPR e dei Cas combinati
merged_df = CRISPR_df.drop(columns=['spacers', 'repeats']).reset_index().merge(Cas_df, on=['MAG', 'Contig'], how="inner", suffixes=('_CRISPR', '_Cas')).set_index('index')

# Aggiunge le colonne al DataFrame
CRISPR_df['Cas_0-1000']=0
CRISPR_df['Cas_1000-10000']=0
CRISPR_df['Cas_>100000']=0
CRISPR_df['Cas_overlayed']=0

# Calcola la distanza tra i CRISPR e i Cas
for index, row in merged_df.iterrows():
    if row['Start_Cas'] >= row['End_CRISPR']:
        # print('Cas davati al CRISPR')
        distance = row['Start_Cas'] - row['End_CRISPR']
    elif row['End_Cas'] <= row['Start_CRISPR']:
        # print('Cas prima il CRISPR')
        distance = row['Start_CRISPR'] - row['End_Cas']
    else:
        # print('Cas che sovrappone al CRISPR')
        distance = -1
    
    if distance >= 0 and distance <= 1000:
        CRISPR_df.at[index, 'Cas_0-1000'] += 1
    elif distance > 1000 and distance <= 10000:
        CRISPR_df.at[index, 'Cas_1000-10000'] += 1
    elif distance > 10000:
        CRISPR_df.at[index, 'Cas_>100000'] += 1
    elif distance == -1:
        CRISPR_df.at[index, 'Cas_overlayed'] += 1
    else:
        print('Errore')
        print('Distanza: ', distance)

# Salva il DataFrame in un file TSV
CRISPR_df.to_csv('/home/isacco.cenacchi/data/Tirocinio/out/MAGs_minced_parsed_cas_real.tsv', sep='\t', index=False)
